In [49]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [53]:
url_base = 'https://boards.greenhouse.io'
companies = ['github', 'gitlab', 'invision', 'blockchain', 'automatticcareers']
# jobs = pd.DataFrame()

for company in companies:
    
    company_url = url_base + '/' + company
    response = requests.get(company_url)
    response_html = BeautifulSoup(response.content, 'html.parser')
    results = response_html.find_all('div', class_='opening')
    
    for result in results:
        job = result.find('a')
        job_url = job['href']
        job_name = job.text
        job_location = result.find('span', class_='location').text
        
        # job description
        job_description_url = 'https://boards.greenhouse.io' + job_url
        job_description_response = requests.get(job_description_url)
        job_description_html = BeautifulSoup(job_description_response.content, 'html.parser')
        job_description = job_description_html.find('div', id='content')
        
        
        jobs_temp = pd.DataFrame({
            'company': [company],
            'job_name': [job_name],
            'location': [job_location],
            'job_url': [job_url],
            'job_description': [job_description]
        })
        
        jobs = pd.concat([jobs, jobs_temp], ignore_index=True)
        
        time.sleep(5)

In [51]:
jobs.head()

,company,job_name,location,job_url,job_description
0,github,IT Product Manager,Remote - US West Coast,/github/jobs/4147448,"[\n, [[We are currently hiring an IT Product M..."
1,github,IT Software Engineer,Remote - US West Coast,/github/jobs/4016529,"[\n, [[The IT Engineering team at GitHub is gr..."
2,github,"Design Director – Education, Communities, and ...",Remote - US,/github/jobs/3992208,"[\n, [GitHub is changing the way the world bui..."
3,github,"Design Director – Education, Communities, and ...",Remote - Canada,/github/jobs/3992209,"[\n, [GitHub is changing the way the world bui..."
4,github,"Design Director – Education, Communities, and ...",Remote - US East,/github/jobs/4177775,"[\n, [GitHub is changing the way the world bui..."


In [54]:
len(jobs)

544

In [55]:
jobs.company.unique()

array(['github', 'gitlab', 'invision', 'blockchain', 'automatticcareers'],
      dtype=object)

In [58]:
jobs.to_csv('jobs_list.csv', index=False)